In [3]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()
# Get the value of the environment variable
api_key = os.getenv('GROQ_API_KEY')

In [9]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_core.documents import Document
from langchain_groq import ChatGroq

# Initialize the Groq chat model with the API key
llm = ChatGroq(groq_api_key=api_key, model_name="deepseek-r1-distill-llama-70b")

graph_transformer = LLMGraphTransformer(llm=llm)

In [10]:
text = """
Game of Thrones is an American fantasy drama television series created by David Benioff and D. B. Weiss for HBO. It is an adaptation of A Song of Ice and Fire, a series of high fantasy novels by George R. R. Martin, the first of which is A Game of Thrones. The show premiered on HBO in the United States on April 17, 2011, and concluded on May 19, 2019, with 73 episodes broadcast over eight seasons.

Set on the fictional continents of Westeros and Essos, Game of Thrones has a large ensemble cast and follows several story arcs throughout the course of the show. The first major arc concerns the Iron Throne of the Seven Kingdoms of Westeros through a web of political conflicts among the noble families either vying to claim the throne or fighting for independence from whoever sits on it. The second major arc focuses on the last descendant of the realm's deposed ruling dynasty, who has been exiled to Essos and is plotting to return and reclaim the throne. The third follows the Night's Watch, a military order defending the realm against threats from beyond the Seven Kingdoms' northern border.

Game of Thrones attracted a record viewership on HBO and has a broad, active, and international fan base. Many critics and publications have named the show one of the greatest television series of all time. Critics have praised the series for its acting, complex characters, story, scope, and production values, although its frequent use of nudity and violence (including sexual violence) generated controversy. The final season received significant criticism for its reduced length and creative decisions, with many considering it a disappointing conclusion. The series received 59 Primetime Emmy Awards, the most by a drama series, including Outstanding Drama Series in 2015, 2016, 2018 and 2019. Its other awards and nominations include three Hugo Awards for Best Dramatic Presentation, a Peabody Award, and five nominations for the Golden Globe Award for Best Television Series – Drama.

A prequel series, House of the Dragon, premiered on HBO in 2022. A second prequel currently in production, A Knight of the Seven Kingdoms, is scheduled to debut in 2026.
"""

In [11]:
documents  = [Document(page_content= text)]
graph_doc = await graph_transformer.aconvert_to_graph_documents(documents)

In [12]:
print(f"Nodes: {graph_doc[0].nodes}")
print(f"Relationships: {graph_doc[0].relationships} ")


Nodes: [Node(id='Game Of Thrones', type='Television series', properties={}), Node(id='David Benioff', type='Person', properties={}), Node(id='D. B. Weiss', type='Person', properties={}), Node(id='Hbo', type='Network', properties={}), Node(id='A Song Of Ice And Fire', type='Book series', properties={}), Node(id='George R. R. Martin', type='Person', properties={}), Node(id='A Game Of Thrones', type='Book', properties={}), Node(id='April 17, 2011', type='Date', properties={}), Node(id='May 19, 2019', type='Date', properties={}), Node(id='73 Episodes', type='Count', properties={}), Node(id='8 Seasons', type='Count', properties={}), Node(id='Westeros', type='Location', properties={}), Node(id='Essos', type='Location', properties={}), Node(id='Iron Throne', type='Concept', properties={}), Node(id='Record Viewership', type='Characteristic', properties={}), Node(id='Broad Fan Base', type='Characteristic', properties={}), Node(id='Critical Acclaim', type='Characteristic', properties={}), Node(i

### Visualize Graph

In [13]:
from pyvis.network import Network

def visualize_graph(graph_documents):

    # Create network
    net = Network(height="1200px", width="100%", directed=True,
                      notebook=False, bgcolor="#222222", font_color="white")
    
    nodes = graph_documents[0].nodes
    relationships = graph_documents[0].relationships

    # Build lookup for valid nodes
    node_dict = {node.id: node for node in nodes}
    
    # Filter out invalid edges and collect valid node IDs
    valid_edges = []
    valid_node_ids = set()
    for rel in relationships:
        if rel.source.id in node_dict and rel.target.id in node_dict:
            valid_edges.append(rel)
            valid_node_ids.update([rel.source.id, rel.target.id])


    # Track which nodes are part of any relationship
    connected_node_ids = set()
    for rel in relationships:
        connected_node_ids.add(rel.source.id)
        connected_node_ids.add(rel.target.id)

    # Add valid nodes
    for node_id in valid_node_ids:
        node = node_dict[node_id]
        try:
            net.add_node(node.id, label=node.id, title=node.type, group=node.type)
        except:
            continue  # skip if error

    # Add valid edges
    for rel in valid_edges:
        try:
            net.add_edge(rel.source.id, rel.target.id, label=rel.type.lower())
        except:
            continue  # skip if error

    # Configure physics
    net.set_options("""
            {
                "physics": {
                    "forceAtlas2Based": {
                        "gravitationalConstant": -100,
                        "centralGravity": 0.01,
                        "springLength": 200,
                        "springConstant": 0.08
                    },
                    "minVelocity": 0.75,
                    "solver": "forceAtlas2Based"
                }
            }
            """)
        
    output_file = "knowledge_graph.html"
    net.save_graph(output_file)
    print(f"Graph saved to {os.path.abspath(output_file)}")

    # Try to open in browser
    try:
        import webbrowser
        webbrowser.open(f"file://{os.path.abspath(output_file)}")
    except:
        print("Could not open browser automatically")
        
# Run the function
visualize_graph(graph_doc)

Graph saved to d:\Projects\KnowledgeGraph_App\knowledge_graph.html


### Extract specific types of nodes

In [14]:
allowed_nodes = ["Person", "Organization", "Location", "Award", "Season"]
graph_transformer_nodes_defined = LLMGraphTransformer(llm=llm, allowed_nodes=allowed_nodes)
graph_documents_nodes_defined = await graph_transformer_nodes_defined.aconvert_to_graph_documents(documents)

In [15]:
print(f"Nodes:{graph_doc[0].nodes}")
print(f"Relationships:{graph_doc[0].relationships}")

Nodes:[Node(id='Game Of Thrones', type='Television series', properties={}), Node(id='David Benioff', type='Person', properties={}), Node(id='D. B. Weiss', type='Person', properties={}), Node(id='Hbo', type='Network', properties={}), Node(id='A Song Of Ice And Fire', type='Book series', properties={}), Node(id='George R. R. Martin', type='Person', properties={}), Node(id='A Game Of Thrones', type='Book', properties={}), Node(id='April 17, 2011', type='Date', properties={}), Node(id='May 19, 2019', type='Date', properties={}), Node(id='73 Episodes', type='Count', properties={}), Node(id='8 Seasons', type='Count', properties={}), Node(id='Westeros', type='Location', properties={}), Node(id='Essos', type='Location', properties={}), Node(id='Iron Throne', type='Concept', properties={}), Node(id='Record Viewership', type='Characteristic', properties={}), Node(id='Broad Fan Base', type='Characteristic', properties={}), Node(id='Critical Acclaim', type='Characteristic', properties={}), Node(id

### Extract specific types of relationships

In [16]:
allowed_nodes = ["Person", "Organization", "Location", "Award", "ResearchField"]
allowed_relationships = [
    ("Person", "WORKS_AT", "Organization"),
    ("Person", "SPOUSE", "Person"),
    ("Person", "AWARD", "Award"),
    ("Organization", "IN_LOCATION", "Location"),
    ("Person", "FIELD_OF_RESEARCH", "ResearchField")
]
graph_transformer_rel_defined = LLMGraphTransformer(
  llm=llm,
  allowed_nodes=allowed_nodes,
  allowed_relationships=allowed_relationships
)
graph_documents_rel_defined = await graph_transformer_rel_defined.aconvert_to_graph_documents(documents)

In [17]:
# Visualize graph
visualize_graph(graph_documents_rel_defined)

Graph saved to d:\Projects\KnowledgeGraph_App\knowledge_graph.html
